# Author imformation:
Fan Shengzhe, Shanghaijiaotong University, Shanghai, China  
Email: fanshengzhe@sjtu.edu.cn

# 1.Open Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
os.chdir("drive/My Drive/reco/veg-fru-reco")

In [ ]:
import pandas as pd
log_table = pd.read_csv('./data/log_table.csv')
item_info_raw_table = pd.read_csv('./data/item_table.csv')
log_table

,user_id,item_id,timestamp_ms,environment,device_group,os,province,city,referrer_type,timestamp_s
0,0,160417,1507029570190,0,0,0,0,0,0,1507029570
1,0,5408,1507029571478,0,0,0,0,0,0,1507029571
2,0,50823,1507029601478,0,0,0,0,0,0,1507029601
3,1,157770,1507029532200,0,0,0,0,1,1,1507029532
4,1,96613,1507029671831,0,0,0,0,1,1,1507029671
...,...,...,...,...,...,...,...,...,...,...
1630628,228075,70758,1508211323220,0,1,1,0,1,2,1508211323
1630629,242176,331116,1508211542618,0,1,1,0,1,0,1508211542
1630630,242176,234481,1508211850103,0,1,1,0,1,0,1508211850
1630631,242176,211442,1508212189949,0,1,1,0,1,0,1508212189


# 2.Get timestamp

In [ ]:
import time
def get_timestamp(time_str):
  timeArray = time.strptime(time_str, "%Y-%m-%d %H:%M:%S")
  timeStamp = int(time.mktime(timeArray))
  return timeStamp

def get_timestr(timestamp):
  timeArray = time.localtime(timestamp)
  return time.strftime("%Y-%m-%d %H:%M:%S", timeArray)

In [ ]:
# log_table['timestamp'] = log_table['timestamp'].map(get_timestamp)

# 3.Encode category features

In [ ]:
fea_list = ['user_id', 'environment', 'device_group', 'os', 'province', 'city', 'referrer_type']
for fea in fea_list:
  print(fea, len(log_table[fea].unique()), log_table[fea].isna().sum(), log_table[fea].unique())

print('item_id', len(item_info_raw_table['item_id'].unique()), item_info_raw_table['item_id'].isna().sum(), item_info_raw_table['item_id'].unique())
print('category_id', len(item_info_raw_table['category_id'].unique()), item_info_raw_table['category_id'].isna().sum(), item_info_raw_table['category_id'].unique())

user_id 250000 0 [     0      1      2 ... 249997 249998 249999]
environment 3 0 [0 1 2]
device_group 5 0 [0 1 2 3 4]
os 8 0 [0 1 2 3 4 5 6 7]
province 11 0 [ 0  1  2  3  4  5  6  7  8  9 10]
city 28 0 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]
referrer_type 7 0 [0 1 2 3 4 5 6]
item_id 364047 0 [     0      1      2 ... 364044 364045 364046]
category_id 461 0 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 1

## 3.1 定义encode函数

In [ ]:
def label_encode(log_table, fea_name):
  fea_dict = {}
  for index, raw_value in enumerate(sorted(log_table[fea_name].unique())):
    fea_dict[raw_value] = index

  def encode(raw_value):
    return fea_dict[raw_value]

  log_table[fea_name] = log_table[fea_name].map(encode)

## 3.2 开始encode

In [ ]:
cate_feas = ['user_id', 'environment', 'device_group', 'os', 'province', 'city', 'referrer_type']
for fea_name in cate_feas:
  label_encode(log_table, fea_name)
label_encode(item_info_raw_table, 'category_id')

In [ ]:
for fea in fea_list:
  print(fea, len(log_table[fea].unique()), log_table[fea].isna().sum(), log_table[fea].unique())

user_id 250000 0 [     0      1      2 ... 249997 249998 249999]
environment 3 0 [0 1 2]
device_group 5 0 [0 1 2 3 4]
os 8 0 [0 1 2 3 4 5 6 7]
province 11 0 [ 0  1  2  3  4  5  6  7  8  9 10]
city 28 0 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]
referrer_type 7 0 [0 1 2 3 4 5 6]


## 3.3 单独处理item id

需要注意的一点是，不是所有的item都有交互。item_id来自于log_table、item_info_raw_table、item_emb_table三者的并集。

In [ ]:
item_emb_table = pd.read_csv('./data/item_emb_table.csv')

In [ ]:
infoset = set(item_info_raw_table['item_id'].unique())
logset = set(log_table['item_id'].unique())
embset = set(item_emb_table['item_id'].unique())
print(len(infoset), len(logset), len(embset))

364047 35380 364047


In [ ]:
print(len(logset.difference(infoset)))
print(len(infoset.difference(embset)))
print(len(infoset.difference(logset)))

0
0
328667


In [ ]:
iid_dict = {}
for index, raw_value in enumerate(sorted(item_info_raw_table['item_id'].unique())):
  iid_dict[raw_value] = index

def encode_iid(raw_value):
  return iid_dict[raw_value]

log_table['item_id'] = log_table['item_id'].map(encode_iid)
item_emb_table['item_id'] = item_emb_table['item_id'].map(encode_iid)
item_info_raw_table['item_id'] = item_info_raw_table['item_id'].map(encode_iid)

# 4.save result

In [ ]:
len(log_table['user_id'].unique())

250000

In [ ]:
log_table['timestamp_s'] = log_table['timestamp_s'].map(get_timestr)
item_info_raw_table['prod_at_ts_s'] = item_info_raw_table['prod_at_ts_s'].map(get_timestr)

In [ ]:
log_table.head()

,user_id,item_id,timestamp_ms,environment,device_group,os,province,city,referrer_type,timestamp_s
0,0,160417,1507029570190,0,0,0,0,0,0,2017-10-03 11:19:30
1,0,5408,1507029571478,0,0,0,0,0,0,2017-10-03 11:19:31
2,0,50823,1507029601478,0,0,0,0,0,0,2017-10-03 11:20:01
3,1,157770,1507029532200,0,0,0,0,1,1,2017-10-03 11:18:52
4,1,96613,1507029671831,0,0,0,0,1,1,2017-10-03 11:21:11


In [ ]:
log_table.describe()

,user_id,item_id,timestamp_ms,environment,device_group,os,province,city,referrer_type
count,1.630633e+06,1.630633e+06,1.630633e+06,1.630633e+06,1.630633e+06,1.630633e+06,1.630633e+06,1.630633e+06,1.630633e+06
mean,1.238712e+05,1.947252e+05,1.507524e+12,4.930294e-02,4.134732e-01,5.975710e-01,1.551649e-01,6.628157e+00,1.274414e+00
std,8.188783e+04,9.147543e+04,3.598953e+08,3.076404e-01,5.603972e-01,8.892703e-01,8.826908e-01,6.650579e+00,1.071705e+00
min,0.000000e+00,3.000000e+00,1.506959e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.613700e+04,1.241770e+05,1.507218e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,1.168770e+05,2.023810e+05,1.507562e+12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,2.000000e+00
75%,2.084160e+05,2.771070e+05,1.507767e+12,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+01,2.000000e+00
max,2.499990e+05,3.640460e+05,1.510603e+12,2.000000e+00,4.000000e+00,7.000000e+00,1.000000e+01,2.700000e+01,6.000000e+00


In [ ]:
if not os.path.exists('./cache'):
  os.mkdir('./cache')

In [ ]:
log_table.to_csv('./cache/log_table.csv', index=False)

In [ ]:
item_info_raw_table.to_csv('./cache/item_info_raw_table.csv', index=False)